In [1]:
from NeuralNetwork.dense import Dense
from NeuralNetwork.losses import mse,mse_derive,binary_crossentropy,binary_crossentropy_derive
from NeuralNetwork.activations import Sigmoid,Softmax
from NeuralNetwork.network import predict
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import struct
import matplotlib.pyplot as plt
from PIL import Image



In [5]:
EPOCHS = 50
ALPHA = 0.1

NN = [Dense(784,20),
      Sigmoid(),
      Dense(20,10),
      Softmax()
      ]


def train(NN, loss, loss_derive, X, Y,x_test,y_test, epoch, alpha, verbose = True):
    error_TS = []
    acc_TS = []
    val_error_TS = []
    val_acc_TS = []
    for e in range(epoch):
        errors= 0
        acc =0
        val_error = 0
        val_acc = 0
        for x, y in zip(X, Y):
            # forward
            out = predict(NN, x)
            if np.argmax(y)==np.argmax(out):
                acc+=1
            # error

            errors+= loss(y, out)

            # backward
            gradient = loss_derive(y, out)
            for layer in reversed(NN):
                gradient = layer.backward(gradient, alpha)

        for x, y in zip(x_test, y_test):
            output = predict(NN, x)
            if np.argmax(y)==np.argmax(output):
                val_acc+=1
            val_error += loss(y, output)
        
        errors/= len(X)
        acc /=len(X)
        val_error /= len(x_test)
        val_acc /=len(x_test)

        if verbose:
            print(f"{e + 1}/{epoch}, error={round(errors,4)}, accuracy={round(acc,4)}, val_error={round(val_error,4)}, val_accuracy={round(val_acc,4)}")
        error_TS.append(errors)
        acc_TS.append(acc)
        val_error_TS.append(val_error)
        val_acc_TS.append(val_acc)
    return [error_TS,acc_TS,val_error_TS,val_acc_TS]

In [3]:
def load_mnist_images_to_df(filename):
    with open(filename, 'rb') as f:
        _ = struct.unpack('>I', f.read(4))[0]
        num_images = struct.unpack('>I', f.read(4))[0]
        num_rows = struct.unpack('>I', f.read(4))[0]
        num_cols = struct.unpack('>I', f.read(4))[0]
        image_data = f.read(num_images * num_rows * num_cols)
        images = np.frombuffer(image_data, dtype=np.uint8)
        images = images.reshape(num_images, num_rows * num_cols)
        df = pd.DataFrame(images)
    return df

def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        magic = struct.unpack('>I', f.read(4))[0]
        num_labels = struct.unpack('>I', f.read(4))[0]
        label_data = f.read(num_labels)
        labels = np.frombuffer(label_data, dtype=np.uint8)        
    return labels


In [4]:
X_train = load_mnist_images_to_df('./dataset/train-images.idx3-ubyte')
Y_train = load_mnist_labels('./dataset/train-labels.idx1-ubyte')
X_test = load_mnist_images_to_df('./dataset/t10k-images.idx3-ubyte')
Y_test = load_mnist_labels('./dataset/t10k-labels.idx1-ubyte')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [9]:
X_train.shape

(60000, 784)

In [19]:
df_train = pd.read_csv("./dataset/MNIST.csv")

Y_test,X_test =  df_train.iloc[:2000, :1], df_train.iloc[:2000, 1:]
Y_train,X_train = df_train.iloc[2000:, :1], df_train.iloc[2000:, 1:]

In [10]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [18]:
scaler = StandardScaler()
pca =PCA(0.90)

In [28]:
X_train_scaled = scaler.fit_transform(X_train)
X_train_pca = pca.fit_transform(X_train_scaled,Y_train)
X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)

In [29]:
X_train_pca.shape


(60000, 236)

In [6]:
Y_train =np.squeeze(np.eye(10)[Y_train])
Y_test = np.squeeze(np.eye(10)[Y_test])

In [7]:
#extra
X_in = np.reshape(X_train,X_train.shape +(1,))
X_out = np.reshape(X_test,X_test.shape +(1,))


In [10]:
Y_in = np.reshape(Y_train,Y_train.shape +(1,))
Y_out = np.reshape(Y_test,Y_test.shape +(1,))

In [9]:
X_in = np.reshape(X_train_pca,X_train_pca.shape +(1,))
Y_in = np.reshape(Y_train,Y_train.shape +(1,))
X_out = np.reshape(X_test_pca,X_test_pca.shape +(1,))
Y_out = np.reshape(Y_test,Y_test.shape +(1,))

NameError: name 'X_train_pca' is not defined

In [11]:
metrics = train(NN, binary_crossentropy, binary_crossentropy_derive, X=X_in,Y=Y_in,x_test=X_out,y_test=Y_out, epoch=EPOCHS, alpha=ALPHA)

metrics_names = ['Training Loss','Training Accuracy','Test Loss','Test Accuracy']
for i in range(len(metrics_names)):
    plt.plot(metrics[i])
    plt.xlabel("Epochs")
    plt.ylabel(metrics_names[i])
    plt.savefig("./graph/MNIST_{}.png".format(metrics_names[i]))
    plt.show()

1/50, error=0.0817, accuracy=0.8535, val_error=0.0809, val_accuracy=0.8641
2/50, error=0.0548, accuracy=0.9083, val_error=0.0662, val_accuracy=0.89
3/50, error=0.0481, accuracy=0.9199, val_error=0.0488, val_accuracy=0.9215
4/50, error=0.0448, accuracy=0.9266, val_error=0.0454, val_accuracy=0.9256
5/50, error=0.0429, accuracy=0.9277, val_error=0.0441, val_accuracy=0.9304
6/50, error=0.0406, accuracy=0.9325, val_error=0.0422, val_accuracy=0.9328
7/50, error=0.04, accuracy=0.9333, val_error=0.0414, val_accuracy=0.9316
8/50, error=0.0382, accuracy=0.9364, val_error=0.0559, val_accuracy=0.9163
9/50, error=0.0371, accuracy=0.9391, val_error=0.0428, val_accuracy=0.9336
10/50, error=0.0362, accuracy=0.9408, val_error=0.0456, val_accuracy=0.9279
11/50, error=0.0347, accuracy=0.9424, val_error=0.0428, val_accuracy=0.9304
12/50, error=0.0343, accuracy=0.9428, val_error=0.0411, val_accuracy=0.937


KeyboardInterrupt: 